In [8]:
import server
import requests
import os
import random

In [9]:
os.environ['FLASK_CONFIG'] = './local/server-config-dev.yaml'
app = server.create_app()

In [10]:
models = server.models
db = server.models.db

In [12]:
with app.app_context():
    q = db.session.query(models.Tag)
    q = q.filter_by(has_explicit=True)
    tags = q.all()

tag_names = sorted([t.tag for t in tags])

/Users/zen/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [13]:
tag_names

['apps',
 'aws',
 'containers',
 'docker',
 'healthtech',
 'hippa',
 'jill',
 'tutorial']

In [14]:
def k_unique(vals, k=3):
    output = set()
    while len(output) < k:
        output.add(random.choice(vals))
    return list(output)

In [17]:
search_tags = k_unique(tag_names, k=2)

with app.app_context():
    q = db.session.query(models.Post, db.func.count(models.Tag.tag))
    q = q.select_from(models.Post)
    q = q.join(models.Post.post_tags)
    q = q.join(models.PostTag.tag)
    q = q.filter(models.Tag.tag.in_(search_tags))
    q = q.group_by(models.Post)
#     q = q.having(db.func.count(models.Tag.tag) >= len(search_tags))
    q = q.having(db.func.count(models.Tag.tag) >= 1)
    
    results = q.all()
    output = []
    for r in results:
        post, count = r
        post_tags = [p.tag.tag.lower() for p in post.post_tags]
        post_tags.sort()
        output.append((post, count, post_tags))

In [20]:
print('search:')
print('\t{}'.format(search_tags))
print('results:')
for post, count, post_tags in output:
    msg = '\tpost: {id}, count: {count} <- {tags}'.format(
        id = post.id,
        count = count,
        tags = ', '.join(post_tags)
    )
    print(msg)

search:
	['hippa', 'apps']
results:
	post: 2, count: 1 <- 1, healthtech, hippa, jill, post
	post: 3, count: 1 <- 2, apps, docker, healthtech, post
	post: 7, count: 2 <- 6, apps, healthtech, hippa, post
	post: 8, count: 1 <- 7, apps, aws, post, tutorial
	post: 9, count: 1 <- 8, apps, containers, post, tutorial
	post: 10, count: 1 <- 9, containers, hippa, jill, post
